In [4]:
#This note book failed to run as, it wasn't possible to import qlinear from ipynb kernel
import subprocess

# Define the path to the batch file
batch_file_path = r"C:\Users\mikuv\Desktop\ryzen-ai-sw-1.1\RyzenAI-SW\example\transformers\setup.bat"

# Run the batch file
result = subprocess.run(batch_file_path, shell=True, capture_output=True, text=True)

# Print the output and any errors
print("Output:", result.stdout)
print("Errors:", result.stderr)

Output: 
(ryzenai-transformers) c:\Users\mikuv\Desktop\repos\vegi\ocr\transformers\donut_optimum>SET PWD=C:\Users\mikuv\Desktop\ryzen-ai-sw-1.1\RyzenAI-SW\example\transformers\ 

(ryzenai-transformers) c:\Users\mikuv\Desktop\repos\vegi\ocr\transformers\donut_optimum>SET THIRD_PARTY=C:\Users\mikuv\Desktop\ryzen-ai-sw-1.1\RyzenAI-SW\example\transformers\\third_party 

(ryzenai-transformers) c:\Users\mikuv\Desktop\repos\vegi\ocr\transformers\donut_optimum>SET TVM_LIBRARY_PATH=C:\Users\mikuv\Desktop\ryzen-ai-sw-1.1\RyzenAI-SW\example\transformers\\third_party\lib;C:\Users\mikuv\Desktop\ryzen-ai-sw-1.1\RyzenAI-SW\example\transformers\\third_party\bin 

(ryzenai-transformers) c:\Users\mikuv\Desktop\repos\vegi\ocr\transformers\donut_optimum>SET PATH=c:\Users\mikuv\miniconda3\envs\ryzenai-transformers;C:\Program Files (x86)\Microsoft Visual Studio\2019\Community\Common7\IDE\\Extensions\Microsoft\IntelliCode\CLI;C:\Program Files (x86)\Microsoft Visual Studio\2019\Community\VC\Tools\MSVC\14.29.3

In [3]:
# %pip install C:\Users\mikuv\Desktop\ryzen-ai-sw-1.1\RyzenAI-SW\example\transformers\ops\cpp --force-reinstall

Processing c:\users\mikuv\desktop\ryzen-ai-sw-1.1\ryzenai-sw\example\transformers\ops\cpp
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build RyzenAI
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Building wheel for RyzenAI (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [88 lines of output]
      running bdist_wheel
      running build
      running build_ext
      -- Selecting Windows SDK version 10.0.19041.0 to target Windows 10.0.22631.
      -- The C compiler identification is MSVC 19.29.30154.0
      -- The CXX compiler identification is MSVC 19.29.30154.0
      -- Detecting C compiler ABI info
      -- Detecting C compiler ABI info - done
      -- Check for working C compiler: C:/Program Files (x86)/Microsoft Visual Studio/2019/Community/VC/Tools/MSVC/14.29.30133/bin/Hostx64/x64/cl.exe - skipped
      -- Detecting C compile features
      -- Detecting C compile features - done
      -- Detecting CXX compiler ABI info
      -- Detecting CXX compiler ABI info - done
      -- Check for working CXX compiler: C:/Program Files (x86)/Microsoft Visual Studio/2019/Community/VC/Tools/MSVC/14.29.30133/bin/Hostx64/x64/cl.

In [9]:
import re
import torch
from transformers import DonutProcessor, VisionEncoderDecoderModel, AutoTokenizer
from PIL import Image
import torch
import qlinear 
from utils import Utils
from timeit import default_timer as timer

ModuleNotFoundError: No module named 'qlinear'

In [ ]:
# Load the processor and model
processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base-finetuned-cord-v2")
model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base-finetuned-cord-v2")
tokenizer = AutoTokenizer.from_pretrained("naver-clova-ix/donut-base-finetuned-cord-v2")

In [ ]:
# Extract the encoder and decoder
encoder = model.encoder
decoder = model.decoder

# Quantize the encoder and decoder
quantized_encoder = torch.ao.quantization.quantize_dynamic(
    encoder, {torch.nn.Linear}, dtype=torch.qint8, inplace=True
)
quantized_decoder = torch.ao.quantization.quantize_dynamic(
    decoder, {torch.nn.Linear}, dtype=torch.qint8, inplace=True
)

In [ ]:
# Save the quantized models
torch.save(quantized_encoder, "./torch_q_models/quantized_encoder.pt")
torch.save(quantized_decoder, "./torch_q_models/quantized_decoder.pt")

print("Quantization complete and models saved.")

In [ ]:
#Load quantized models from local
quantized_encoder = torch.load("./torch_q_models/quantized_encoder.pt")
quantized_decoder = torch.load("./torch_q_models/quantized_decoder.pt")

In [ ]:
# Replace the quantized linear layers with QLinear in both encoder and decoder
node_args = ()
node_kwargs = {'device': 'aie'}
# Utils.replace_node(quantized_encoder, torch.ao.nn.quantized.dynamic.modules.linear.Linear, qlinear.QLinear, node_args, node_kwargs)
Utils.replace_node(quantized_decoder, torch.ao.nn.quantized.dynamic.modules.linear.Linear, qlinear.QLinear, node_args, node_kwargs)
# print(quantized_encoder)
# print(quantized_decoder)

In [ ]:
for i in range(1, 6):
    #Start Timer:
    start = timer()

    # Load the image from a local file
    image_path = ".\\images\\test_image_"+str(i)+".png"  # Update this to the path of your image
    image = Image.open(image_path).convert('RGB')

    # Prepare decoder inputs
    task_prompt = "<s_synthdog>"
    decoder_input_ids = tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt").input_ids

    # Preprocess the image
    pixel_values = processor(image, return_tensors="pt")
    # print(pixel_values.shape)

    # Perform encoding with debug statements
    with torch.no_grad():
        encoder_outputs = quantized_encoder(**pixel_values)
        # print("Encoder outputs shape:", encoder_outputs.last_hidden_state.shape)

    # Initialize the decoder input for the first time step
    generated_ids = decoder_input_ids

    # Perform decoding step-by-step
    max_length = 512  # Maximum length of the output sequence
    for _ in range(max_length):
        # print(i)
        with torch.no_grad():
            outputs = quantized_decoder(input_ids=generated_ids, encoder_hidden_states=encoder_outputs.last_hidden_state)
            # print("Decoder outputs shape:", outputs.logits.shape)
        next_token_logits = outputs.logits[:, -1, :]
        next_token_id = torch.argmax(next_token_logits, dim=-1, keepdim=True)
        generated_ids = torch.cat([generated_ids, next_token_id], dim=-1)
        if next_token_id.item() == tokenizer.eos_token_id:
            break

    # Decode the generated token IDs to get the output text
    sequence = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    sequence = re.sub(r"<.*?>", "", sequence, count=1).strip()  # Remove first task start token

    # Convert the sequence to JSON
    output_json = processor.token2json(sequence)
    print(output_json)
    #End Timer
    # print(f"CPU Total Time: {timer() - start}")
    print(f"NPU Total Time: {timer() - start}")
